In [1]:
import numpy as np
import fitsio
import glob
import proplot as pplt
import scipy

import joblib
import ngmix

In [2]:
fnames = sorted(glob.glob("blinded_data/*.fits"))

In [35]:
def _comp_weight(g1, g2, gcov11, gcov22, sn):
    err_term = 0.5 * (gcov11 + gcov22)
    weights = 1.0/(sn**2 + err_term)
    prior = ngmix.priors.GPriorBA(0.3, rng=np.random.RandomState())
    pvals = prior.get_prob_array2d(g1, g2)
    # weights *= pvals

    return weights

def _comp_mags(flux, i):
    zp = 30.0
    b_array = np.array([3.27e-12, 4.83e-12, 6.0e-12, 9.0e-12])
    bscale = np.array(b_array) * 10.**(zp / 2.5)
    mag = 2.5 * np.log10(1.0 / b_array[i]) - np.arcsinh(0.5 * flux / bscale[i]) / (0.4 * np.log(10.0))
    # mag_err = 2.5 * fluxerr / (2.0 * bscale[i] * np.log(10.0) * np.sqrt(1.0 + (0.5 * flux / bscale[i])**2.))    
    return mag


# copied from Masa
def cut_and_weight(d, mdet_mom, sn):
    mag_g = _comp_mags(d[mdet_mom + "_band_flux_g"], 0)
    mag_r = _comp_mags(d[mdet_mom + "_band_flux_r"], 1)
    mag_i = _comp_mags(d[mdet_mom + "_band_flux_i"], 2)
    mag_z = _comp_mags(d[mdet_mom + "_band_flux_z"], 3)
    gmr = mag_g - mag_r
    rmi = mag_r - mag_i
    imz = mag_i - mag_z

    msk = (
        (d[mdet_mom + "_flags"] == 0) & 
        (d["mask_flags"] == 0) & 
        (d["shear_bands"] == '0123') & 
        (d[mdet_mom+"_band_flux_flags_g"] == 0) & 
        (d[mdet_mom+"_band_flux_flags_r"] == 0) & 
        (d[mdet_mom+"_band_flux_flags_i"] == 0) & 
        (d[mdet_mom+"_band_flux_flags_z"] == 0) & 
        (d[mdet_mom+"_s2n"] > 10) & 
        (d["mfrac"] < 0.1) &
        (np.abs(gmr) < 5) & 
        (np.abs(rmi) < 5) & 
        (np.abs(imz) < 5) & 
        np.isfinite(mag_g) & 
        np.isfinite(mag_r) &
        np.isfinite(mag_i) & 
        np.isfinite(mag_z) & 
        (mag_g < 26.5) & 
        (mag_r < 26.5) & 
        (mag_i < 26.2) & 
        (mag_z < 25.6))
    
    if mdet_mom == 'wmom':
        msk_add = ((d[mdet_mom + "_T_ratio"] > 1.2))
        msk = msk & msk_add
    elif mdet_mom == 'pgauss':
        msk_add = ((d[mdet_mom + "_T_ratio"] > 0.5) & (d[mdet_mom + "_T"] < 1.9 - 2.8*d[mdet_mom + "_T_err"]))
        msk = msk & msk_add
    elif mdet_mom == 'pgauss_reg0.90':
        msk_add = ((d[mdet_mom + "_T_ratio"] > 0.5) & (d[mdet_mom + "_T"] < 1.9 - 2.8*d[mdet_mom + "_T_err"]))
        msk = msk & msk_add
    
    return d[msk], _comp_weight(
        d[mdet_mom + "_g_1"][msk], 
        d[mdet_mom + "_g_2"][msk], 
        d[mdet_mom + "_g_cov_1_1"][msk],
        d[mdet_mom + "_g_cov_2_2"][msk],
        sn,
    )

In [41]:
def _compute_sums_g(d, wgt, mdet_mom):
    def _step_msk(step):
        smsk = (
            (d["mdet_step"] == step)
        )
        return smsk
        
    msk = _step_msk("noshear")
    g1_noshear = np.sum(d[mdet_mom + "_g_1"][msk] * wgt[msk])
    wgt_noshear = np.sum(wgt[msk])

    msk = _step_msk("1p")
    g1_1p = np.sum(d[mdet_mom + "_g_1"][msk] * wgt[msk])
    wgt_1p = np.sum(wgt[msk])

    msk = _step_msk("1m")
    g1_1m = np.sum(d[mdet_mom + "_g_1"][msk] * wgt[msk])
    wgt_1m = np.sum(wgt[msk])
    
    return (g1_noshear, wgt_noshear, g1_1p, wgt_1p, g1_1m, wgt_1m)

In [46]:
def _process_file(fname):
    try:
        d = fitsio.read(fname)
    except Exception:
        return None
    bin_msk = (
        (d["psfrec_g_1"] > -0.01)
        & 
        (d["psfrec_g_2"] < 0.01)
    )
    d = d[bin_msk]
    d, wgt = cut_and_weight(d, "pgauss", 0.25)
    sums = _compute_sums_g(d, wgt, "pgauss")
    return sums

In [ ]:
jobs = [
    joblib.delayed(_process_file)(fname)
    for fname in fnames
]

with joblib.Parallel(n_jobs=32, verbose=50) as par:
    all_sums = par(jobs)
    
all_sums = [a for a in all_sums if a is not None]

[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.


/gpfs02/astro/workarea/beckermr/miniconda3/envs/desy6/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[Parallel(n_jobs=32)]: Done   1 tasks      | elapsed:  2.4min
[Parallel(n_jobs=32)]: Done   2 tasks      | elapsed:  2.4min
[Parallel(n_jobs=32)]: Done   3 tasks      | elapsed:  2.4min
[Parallel(n_jobs=32)]: Done   4 tasks      | elapsed:  2.4min
[Parallel(n_jobs=32)]: Done   5 tasks      | elapsed:  2.4min
[Parallel(n_jobs=32)]: Done   6 tasks      | elapsed:  2.4min
[Parallel(n_jobs=32)]: Done   7 tasks      | elapsed:  2.4min
[Parallel(n_jobs=32)]: Done   8 tasks      | elapsed:  2.4min
[Parallel(n_jobs=32)]: Done   9 tasks      | elapsed:  2.4min
[Parallel(n_jobs=32)]: Done  10 tasks      | elapsed:  2.4min
[Parallel(n_jobs=32)]: Done  11 tasks      | elapsed:  2.4min
[Parallel(n_jobs=32)]: Done  12 tasks      | elapsed:  2.4min
[Parallel(n_jobs=32)]: Done  13 tasks      | elapsed:  2.4min
[Parallel(n_jobs=32)]: Done  14 tasks      | elapsed:  2.4min
[Parallel(n_jobs=32)]: Done  15 tasks      | elapsed:  2.4min
[Parallel(n_jobs=32)]: Done  16 tasks      | elapsed:  2.4min
[Paralle

[Parallel(n_jobs=32)]: Done 134 tasks      | elapsed:  4.3min
[Parallel(n_jobs=32)]: Done 135 tasks      | elapsed:  4.3min
[Parallel(n_jobs=32)]: Done 136 tasks      | elapsed:  4.3min
[Parallel(n_jobs=32)]: Done 137 tasks      | elapsed:  4.3min
[Parallel(n_jobs=32)]: Done 138 tasks      | elapsed:  4.4min
[Parallel(n_jobs=32)]: Done 139 tasks      | elapsed:  4.4min
[Parallel(n_jobs=32)]: Done 140 tasks      | elapsed:  4.4min
[Parallel(n_jobs=32)]: Done 141 tasks      | elapsed:  4.4min
[Parallel(n_jobs=32)]: Done 142 tasks      | elapsed:  4.4min
[Parallel(n_jobs=32)]: Done 143 tasks      | elapsed:  4.4min
[Parallel(n_jobs=32)]: Done 144 tasks      | elapsed:  4.5min
[Parallel(n_jobs=32)]: Done 145 tasks      | elapsed:  4.5min
[Parallel(n_jobs=32)]: Done 146 tasks      | elapsed:  4.5min
[Parallel(n_jobs=32)]: Done 147 tasks      | elapsed:  4.5min
[Parallel(n_jobs=32)]: Done 148 tasks      | elapsed:  4.5min
[Parallel(n_jobs=32)]: Done 149 tasks      | elapsed:  4.5min
[Paralle

[Parallel(n_jobs=32)]: Done 268 tasks      | elapsed:  6.0min
[Parallel(n_jobs=32)]: Done 269 tasks      | elapsed:  6.0min
[Parallel(n_jobs=32)]: Done 270 tasks      | elapsed:  6.0min
[Parallel(n_jobs=32)]: Done 271 tasks      | elapsed:  6.0min
[Parallel(n_jobs=32)]: Done 272 tasks      | elapsed:  6.0min
[Parallel(n_jobs=32)]: Done 273 tasks      | elapsed:  6.0min
[Parallel(n_jobs=32)]: Done 274 tasks      | elapsed:  6.0min
[Parallel(n_jobs=32)]: Done 275 tasks      | elapsed:  6.0min
[Parallel(n_jobs=32)]: Done 276 tasks      | elapsed:  6.1min
[Parallel(n_jobs=32)]: Done 277 tasks      | elapsed:  6.1min
[Parallel(n_jobs=32)]: Done 278 tasks      | elapsed:  6.1min
[Parallel(n_jobs=32)]: Done 279 tasks      | elapsed:  6.1min
[Parallel(n_jobs=32)]: Done 280 tasks      | elapsed:  6.1min
[Parallel(n_jobs=32)]: Done 281 tasks      | elapsed:  6.1min
[Parallel(n_jobs=32)]: Done 282 tasks      | elapsed:  6.1min
[Parallel(n_jobs=32)]: Done 283 tasks      | elapsed:  6.1min
[Paralle

In [ ]:
def _jk_est(all_sums):
    all_sums = np.array(all_sums)
    n = all_sums.shape[0]
    gjk = np.zeros(n)
    inds = np.arange(n)
    for i in range(n):
        msk = inds != i
        sums = np.sum(all_sums[msk, :], axis=0)
        g1 = sums[0]/sums[1]
        g1_1p = sums[2]/sums[3]
        g1_1m = sums[4]/sums[5]
        R11 = (g1_1p - g1_1m)/0.02

        gjk[i] = g1/R11
        
    mn = np.mean(gjk)
    dgjk = gjk - mn
    sd = np.sqrt((n-1)/n*np.sum(dgjk**2))
    return mn, sd

_jk_est(all_sums)

In [45]:
_jk_est(all_sums)

(0.00039565560346909806, 0.00021446005657011638)

In [34]:
_jk_est(all_sums)

(0.0002184235401363807, 9.72159928586841e-05)

In [25]:
_jk_est(all_sums)

(-0.00014460306345271345, 0.000235431127251504)

In [ ]:
sums.shape

In [8]:
all_sums

[(4.148640254282828,
  409.58904498494064,
  3.905843849807208,
  405.05307310157224,
  1.2850200429583065,
  378.1445046584588),
 (5.190489715743471,
  2707.8555208123053,
  20.787107623789502,
  2560.8336720602083,
  -26.787899126501376,
  2549.947950121388),
 (-88.90207681868523,
  18705.715228021712,
  21.688148580576893,
  17403.531252705638,
  -164.66104780769086,
  17456.293310327892),
 (-8.497960626558747,
  8124.47181762908,
  6.944340555409674,
  7660.273417068771,
  -20.362422706275787,
  7528.155502146057),
 (-11.907084805109717,
  744.9705288729995,
  -10.423498606195427,
  659.7937852372324,
  -15.12032055269469,
  680.342758036594),
 (-29.951548092735393,
  2522.668803354378,
  -24.87051751117708,
  2421.3420990071645,
  -21.266209202295556,
  2429.5194183705726),
 (2.3516035217358158,
  8.695506995868659,
  2.1529830716634675,
  6.546850580857668,
  2.323901142262385,
  9.54867804717557),
 (-10.054781737332249,
  590.1397857976372,
  -8.208438796651587,
  540.5477225295